In [43]:
import pandas_datareader as pdr
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
def get(tickers, start, end):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker))
    datas = map (data, tickers)
    return(pd.concat(datas, keys = tickers, names = ['Ticker', 'Date']))

In [64]:
tickers = ['^GSPC', '^N225', '000001.SS']
mkt_index = get(tickers)

In [121]:
mkt_index.head(2)

High          Low         Open       Close  \
Ticker Date                                                            
^GSPC  2010-01-04  1133.869995  1116.560059  1116.560059  1132.98999   
       2010-01-05  1136.630005  1129.660034  1132.660034  1136.52002   

                         Volume   Adj Close  
Ticker Date                                  
^GSPC  2010-01-04  3.991400e+09  1132.98999  
       2010-01-05  2.491020e+09  1136.52002

In [133]:
mkt_index_close = mkt_index[['Close']]
mkt_index_returns = mkt_index_close / mkt_index_close.shift(1) - 1
spx_returns = mkt_index_returns.iloc[mkt_index_returns.index.get_level_values('Ticker') == '^GSPC']
spx_returns.index = spx_returns.index.droplevel('Ticker')
nikkei_returns = mkt_index_returns.iloc[mkt_index_returns.index.get_level_values('Ticker') == '^N225']
nikkei_returns.index = nikkei_returns.index.droplevel('Ticker')

In [142]:
spx_returns.to_csv('spx_return.csv')

In [134]:
nikkei_returns.head(2)

,Close
Date,
2010-01-04,2.849580
2010-01-05,0.002538


In [136]:
mkt_index_returns.head(2)

Close
Ticker Date                
^GSPC  2010-01-04       NaN
       2010-01-05  0.003116

In [137]:
spx_returns.head(2)

,Close
Date,
2010-01-04,NaN
2010-01-05,0.003116


In [138]:
nikkei_returns.head(2)

,Close
Date,
2010-01-04,2.849580
2010-01-05,0.002538


In [139]:
return_data = pd.concat([spx_returns, nikkei_returns], axis = 1)
return_data.columns = ['SPX', 'NIKKEI']
return_data

,SPX,NIKKEI
Date,,
2010-01-04,NaN,2.849580
2010-01-05,0.003116,0.002538
2010-01-06,0.000546,0.004645
2010-01-07,0.004001,-0.004640
2010-01-08,0.002882,0.010922
2010-01-11,0.001747,NaN
2010-01-12,-0.009381,0.007484
2010-01-13,0.008326,-0.013246
2010-01-14,0.002426,0.016083
